In [2]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# Load the data from the .mat file
import scipy.io
data = scipy.io.loadmat(r'C:\Users\pezhm\Desktop\DESK\ta\1.mat')

# Extract training data and labels
X = data['Data_Train_ML']
y = data['Label_Train_ML'].ravel()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class RegularizedDiscriminantClassifier:
    def __init__(self, lambda_param=0.1):
        self.lambda_param = lambda_param

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.means_ = {}
        self.cov_ = np.cov(X, rowvar=False)
        self.inv_cov_ = None

        # Regularize the covariance matrix
        self.cov_ += self.lambda_param * np.eye(X.shape[1])
        self.inv_cov_ = np.linalg.inv(self.cov_)

        for cls in self.classes_:
            self.means_[cls] = np.mean(X[y == cls], axis=0)

    def predict(self, X):
        predictions = []
        for sample in X:
            scores = []
            for cls in self.classes_:
                mean_diff = sample - self.means_[cls]
                score = -0.5 * np.dot(np.dot(mean_diff.T, self.inv_cov_), mean_diff)
                scores.append(score)
            predictions.append(self.classes_[np.argmax(scores)])
        return np.array(predictions)

# Define the parameter space for grid search
param_grid = {'lambda_param': [0.1, 0.5, 1, 5, 10]}

# Perform grid search manually
best_accuracy = 0
best_lambda = None
for param in param_grid['lambda_param']:
    rdc = RegularizedDiscriminantClassifier(lambda_param=param)
    rdc.fit(X_train, y_train)
    y_pred = rdc.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Lambda: {param}, Accuracy: {accuracy * 100:.2f}%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_lambda = param

print("Best lambda:", best_lambda)
print("Best model accuracy on test set:", best_accuracy * 100)

Lambda: 0.1, Accuracy: 81.25%
Lambda: 0.5, Accuracy: 81.25%
Lambda: 1, Accuracy: 81.25%
Lambda: 5, Accuracy: 87.50%
Lambda: 10, Accuracy: 87.50%
Best lanbda: 5
Best model accuracy on test set: 87.5
Lambda: 0.1, Accuracy: 81.25%
Lambda: 0.5, Accuracy: 81.25%
Lambda: 1, Accuracy: 81.25%
Lambda: 5, Accuracy: 87.50%
Lambda: 10, Accuracy: 87.50%
Best lambda: 5
Best model accuracy on test set: 87.5
